学生証番号 :

名前 :

メールアドレス(法政大学) :

# B2-3 事前学習済のCNN(VGG16モデル）を用いた画像認識体験


1000クラス120万枚のImageNet画像セットを事前に学習しているVGG16モデルを用いて、画像の認識体験を行う

VGG16モデルは、2014年のILSVRCで優勝した、畳み込み層と全結合層併せて16層からなるCNNモデルである。

<img src="https://drive.google.com/uc?export=view&id=1l4qf_ohJpsagRznuFkt2C8ETK5gilYxp" width = 70%></img>

224$\times$224の画像を受け付けて、それが学習済みの1000クラスのうちどれにあたるかを推定する。





In [ ]:
# [1-0]
# ライブラリのインポート
from tensorflow.keras.applications import xception
from tensorflow.keras.applications import vgg16
from tensorflow.keras.preprocessing import image
import numpy as np

import PIL
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

In [ ]:
# [1-1]
# モデルを呼び出す関数
def get_model(arch="vgg"):
    if arch == "vgg":
        return vgg16.VGG16(weights="imagenet", include_top=True)
    elif arch == "xception":
        return xception.Xception(weights="imagenet", include_top=True)
    else:
        print("ARCH not Found")
        return None


# 画像をリサイズする関数(モデルに合わせて大きさを変更)
def resize(img, arch):
    if arch == "vgg":
        return img.resize((224, 224))
    elif arch == "xception":
        return img.resize((299, 299))
    else:
        print("ARCH not Found")
        return None


# 画像をリサイズし正規化を行う関数(モデルに合わせて変更)
def preprocess_input(img, arch="vgg", use_normalize=True):
    img = resize(img, arch)
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    if use_normalize:
        if arch == "vgg":
            return vgg16.preprocess_input(x)
        elif arch == "xception":
            return xception.preprocess_input(x)
        else:
            print("ARCH not Found")
            return None
    else:
        return x


# 予測された確率から、上位をラベルと一緒に返す関数
def decode_predictions(preds, arch="vgg", top=5):
    if arch == "vgg":
        return vgg16.decode_predictions(preds, top)
    elif arch == "xception":
        return xception.decode_predictions(preds, top)
    else:
        print("ARCH not Found")
        return None

In [ ]:
# [1-2]
# モデルの設定
ARCH = "vgg"
# Xceptionを使用する場合は以下のコメントを外す。
# ARCH = "Xception"

In [ ]:
# [1-3]
# 学習済みのモデルを読み込む
model = get_model(ARCH)
# モデルの概要を表示
model.summary()

最後のprediction (Dense)と書かれているところは、1000種類の認識対象に対応した1000次元のベクトルである。

この表示だけではわからないが、このモデルの出力はsoftmaxになっているので、1000次元の出力は対応する各クラスの確率になっており、1000次元の合計値は1である。


## 自前のデータの用意

認識したい(試したい）画像を用意します。右のタブのフォルダのマークをクリックし、下の画像に従って画像のアップロードを行ってください。

![](https://i.imgur.com/VP6Dfeb.png)

パスのコピーまでできたら以下の `img_path = ` に続く部分にシングルクォートかダブルクォートで挟んでペーストしてください。


In [ ]:
# [1-4]
####################################
# 自分の画像で試したい場合、そのファイルをColab上において指定する。
# 以下にパスを指定する。
img_path = 'DSC_1775.JPG'
img = image.load_img(img_path)
#############################################

# 画像の表示
plt.imshow(img)

In [ ]:
# [1-4]
####################################
# 自分の画像で試したい場合、そのファイルをColab上において指定する。
# 以下にパスを指定する。
img_path = "'
img = image.load_img(img_path)
#############################################

# 画像の表示
plt.imshow(img)

機械的に指定のサイズ(VGGなら224$\times$224)になるので、長方形の画像を入れるとつぶれることに注意。


In [ ]:
# [1-5]
# リサイズ後の画像を表示
resized_img = resize(img, ARCH)
plt.imshow(resized_img)

指定の入力サイズになった画像の各色$x$に対して、事前学習で用いた画像の各チャンネルの平均値で引くという前処理が入っています。

In [ ]:
# [1-6]
# チャンネルごとに平均値を引く処理
x = preprocess_input(img, ARCH)
print(x.shape)

plt.imshow(x[0])

平均値を引いているため全体的に暗くなっている。この変換を行うことで学習通りの性能を実現できる。

次に学習済みのモデルに対して、正規化された画像を入力して認識結果を得る。

In [ ]:
# [1-7]
# 学習済みのモデルを使用し、自前データを予測する。
# predict_on_batchはpredictよりも早いので採用している
output = model.predict_on_batch(x)

結果は1000次元の（numpyの配列形式：numpy.ndarray）ベクトルで、合計が１の確率を表している。

In [ ]:
# [1-8]
# 出力形式を確認
print(type(output))
print(output.shape)

# 最初の10クラスの出力を見てみる
print(output[0, 0:10])

# 合計値を確認
print(f" 1000次元の出力の合計値は　{np.sum(output[0])}")

In [ ]:
# [1-9]
# その結果、推定されたトップ10種類の名前を確率とともに表示。
pred = decode_predictions(output, ARCH, top=10)
print("上位\t予測ラベル \t 確率")
for i, (_, label, prob) in enumerate(pred[0]):
    print(f"{i+1}\t{label.ljust(20)}\t{prob}")

# 課題 B2-3-1 事前学習モデルの性能確認
様々な入力画像を用意し、事前学習モデルの予測結果を確認し、以下について考察せよ。
- 予測が間違えた場合なぜそのような結果になったのか
- 上位の10個に出てきたラベルを見て考えられること

ヒント : 以下のコードを使用することで画像読み込みから、予測の表示までを行える。
```
# 以下の""の間に新しくアップロードした画像のパスを指定
img_path = ""
img = image.load_img(img_path)
# 画像の表示
plt.imshow(img)
# 前処理
x = preprocess_input(img, ARCH)
# 予測結果の確認
output = model.predict_on_batch(x)
pred = decode_predictions(output, ARCH, top=10)
print("上位\t予測ラベル \t 確率")
for i, (_, label, prob) in enumerate(pred[0]):
  print(f"{i+1}\t{label.ljust(20)}\t{prob}")
```

In [ ]:
# ここから実験を追加する。(自由にセルを増やしてOK)

# 課題 B2-3-2 VGGの畳み込み層の構造
畳み込み層最後のConv5_3の畳み込みフィルタにより得られるfeature mapの1つの値(ピクセル)は入力空間でどのくらいの範囲の影響を受けているか？　また、このことからどんなことが考えられるか？（以下の構造図を踏まえて考えよ）

<img src="https://drive.google.com/uc?export=view&id=1l4qf_ohJpsagRznuFkt2C8ETK5gilYxp" width = 70%></img>
